# wordslab-notebooks-lib.jupyterlab

> Access wordslab-notebooks Jupyterlab extension version, current notebook path, json content and cell id, and create or update cells.

## Work together with AI in a Jupyterlab notebook - the Solveit method

A Jupyter notebook is a convenient way to build context for a LLM one cell after the other: you are working in a fully editable conversation, while interacting with AI and with code.

**Jeremy Howard** and his team at **Answer.ai** explored how to work efficiently in this kind of conversation: they developed a method and platform called Solveit.

https://solve.it.com/

We would like to replicate this approach to working with AI in a wordslab notebooks environment. 

Here is how we chose to do it:
- in a jupyterlab notebook, there are two types of cells: markdown and code
- we want to simulate a third type of cell: a "prompt" cell
- the content of this cell is a prompt (text in markdown format) which is sent to an llm when the cell is executed, along with the text of all the cells situated above in the notebook (context)
- the llm response is streamed just below and formatted as markdown.

To simulate this "prompt" cell we need to develop a **Jupyterlab frontend extension** which implements the following behaviors :
- three buttons are added to the cell toolbar: "note", "prompt", "code"
- a click on one of these buttons changes the type of the cell
  - "note" selects a classic markdown cell
  - "prompt" selects a code cell, modified with the special "prompt behavior" defined below
  - "code" selects a classic code cell
- a "prompt" cell is distinguished from a regular code cell by a metadata property registered in the ipynb file
- each cell type is visualized by a specific color in the left border of the cell
  - "note" cell has a green border
  - "prompt" cell a red border
  - "code" cell has a blue border
- the "prompt" cell is a code cell with the specific modified behaviors
  - the code syntax highlighting is replaced by markdown syntax highlighting when the user types text in this cell
  - when the user executes this cell, the frontend extension does the following
    - calls the kernel to inject the following variables
      - __notebook_path with the path and name of the notebook in the workspace
      - __notebook_content with the full json representation of the notebook
      - __cell_id with the id of the current cell
    - then calls the kernel to execute a specific chat(message) python function
      - where the message parameter is the content of the cell
      - and the content of the notebook above the current cell is inluded as context
  - the python chat() function streams the response tokens from the llm to the output section of the code cell, with markdown rendering

See the section "Develop a Jupyterlab frontend extension" at the bottom of this page to understand how the extension was developed.

## Install the Jupyterlab extension - wordslab-notebooks-lib

If you want to use "prompt" cells, you will first need to install the Jupyterlab frontend extension:
- activate your Jupyterlab python virtual environment
- **pip install wordslab-notebooks-lib**
- restart your Jupyterlab server

The extension is **already pre-installed in the wordslab-notebooks environment**.

To be clear: the wordslab-notebooks-lib package contains both: the Javascript Jupyterlab frontend extension AND the python library wich is loaded in the python kernel.

The Jupyterlab frontend extension is reloaded and re-initialized each time you refresh your browser page: 
- to check is the extension is installed and running, look at the browser console and llok for the message 'Wordslab notebooks extension vx.y.z activated'
- hit the refresh button if you encounter a bug and the extension stops working

## Communicate with the Jupyterlab extension

In [47]:
#| default_exp notebook

In [14]:
#| export
import nbformat

In [49]:
#| export
class JupyterlabNotebook:
    def __init__(self):
        if not "__wordslab_extension_version" in globals():
            raise RuntimeError(
                "The JupyterLab extension for wordslab-notebooks is not activated: "
                "please execute `pip install wordslab-notebooks-lib` in JupyterLab virtual environment "
                "and refresh you web browser."
            )
    
    @property
    def jupyterlab_extension_version(self):
        return globals()["__wordslab_extension_version"]
    
    @property
    def path(self):
        return globals()["__notebook_path"]

    @property
    def content(self):
        return nbformat.from_dict(globals()["__notebook_content"])
    
    @property
    def cell_id(self):
        return globals()["__cell_id"]

In [38]:
notebook = JupyterlabNotebook()

In [39]:
notebook.jupyterlab_extension_version

'0.0.11'

In [40]:
notebook.path

'wordslab-notebooks-lib/nbs/02_jupyterlab.ipynb'

In [41]:
notebook.content.metadata

{'kernelspec': {'display_name': 'wordslab-notebooks-lib',
  'language': 'python',
  'name': 'wordslab-notebooks-lib'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.12.12'}}

In [42]:
notebook.cell_id

'd16ad869-d651-40bd-af2c-623d82b4edf0'

In [43]:
notebook.cell_id

'59347d6e-d1f7-4d23-b041-143e42887f6d'

In [50]:
#| hide
import nbdev; nbdev.nbdev_export()

## Develop a Jupyterlab frontend extension

### Understand Jupyterlab kernels and frontend extensions

Jupyter kernels technical implementation details

https://chatgpt.com/share/692bea08-4510-8004-b9ab-c02feeb97c08

Jupyterlab extension development tutorial

https://jupyterlab.readthedocs.io/en/latest/extension/extension_tutorial.html

### Intialize the components of a frontend extension

The source code of the Jupyterlab frontend extension can be found in the following files:

Typescript source code, dependencies, and compilation config:

- `src/index.ts`
- `package.json`
- `tsconfig.json`
- `.yarnrc.yml`

Extension manifest and Javascript compiled code

- wordslab_notebooks_lib/labextension
  - package.json
  - static/remoteEntry.97d57e417eaf8ebadeb6.js 

This is how the extension files are included in the python package:

- `MANIFEST.in` 

```
include install.json
include package.json
recursive-include wordslab_notebooks_lib/labextension *

graft wordslab_notebooks_lib/labextension
graft src
```

This is how the extension files are installed in Jupyterlab extensions directory when the python package is installed:

- `pyproject.toml`

```toml
[tool.setuptools]
include-package-data = true 

[tool.setuptools.data-files]
"share/jupyter/labextensions/wordslab-notebooks-lib" = [
  "wordslab_notebooks_lib/labextension/package.json",
  "install.json"
]
"share/jupyter/labextensions/wordslab-notebooks-lib/static" = [
  "wordslab_notebooks_lib/labextension/static/*"
]
```

This how the command `jupyter labextension develop` finds the directory where the extension files live:

- `wordslab_notebooks_lib\__init__.py`

```python
def _jupyter_labextension_paths():
    return [{
        "src": "labextension",
        "dest": "wordslab-notebooks-lib"
    }]
```

This is how the python package is identified as a Jupyterlab extension in pypi:

- `pyproject.toml`

```
classifiers = [ "Framework :: Jupyter :: JupyterLab :: Extensions :: Prebuilt" ]
```

### Install the Jupyterlab frontend extension in development mode

Open a Terminal

```bash
cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib
source $JUPYTERLAB_ENV/.venv/bin/activate

# Install Javascript dependencies
jlpm install

# Build TypeScript extension
jlpm build

# Register the extension with JupyterLab during development
# jupyter labextension develop . --overwrite
rm $JUPYTERLAB_ENV/.venv/share/jupyter/labextensions/wordslab-notebooks-lib
ln -s $WORDSLAB_WORKSPACE/wordslab-notebooks-lib/wordslab_notebooks_lib/labextension/ $JUPYTERLAB_ENV/.venv/share/jupyter/labextensions/wordslab-notebooks-lib

# Verify extension is found
jupyter labextension list
```

### Test the Jupyterlab frontend extension 

After installing the extension in development mode once, you can iterate fast:
- update the code in `src/index.ts`
- build the extension with `jlpm build`

```bash
cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib
source $JUPYTERLAB_ENV/.venv/bin/activate

# Build TypeScript extension
jlpm build
```
- **refresh** the Jupyterlab single page app in your browser
- test the updated extension

No need to reinstall the extension or to restart Jupyterlab itself, just refresh your browser page.

## Explore the notebook format

https://nbformat.readthedocs.io/en/latest/format_description.html

In [45]:
nb = nbformat.from_dict(__notebook_content)

code_language = nb.metadata.language_info.name
print("> " + code_language + " notebook")

for cell in nb.cells:
    if cell.id == __cell_id: break
        
    is_markdown = cell.cell_type == "markdown"
    is_code = cell.cell_type == "code"
    is_raw = cell.cell_type == "raw"

    print("---------------------")
    print("cell", cell.id, cell.cell_type)
    print("---------------------")
    if is_markdown:
        print(cell.source[:100])
    elif is_code:
        print(f"```{code_language}\n" + cell.source[:100] + "\n```")
    elif is_raw:
        print(cell.source[:100])
    if is_code and cell.execution_count>0 and len(cell.outputs)>0:
        print("---------------------")
        print("cell outputs", cell.id, cell.execution_count)
        print("---------------------")
        for output in cell.outputs:
            if output.output_type == "stream":
                print(f"<{output.name}>")
                print(output.text[:100])
                print(f"</{output.name}>")
            elif output.output_type == "display_data":
                print("<display>")
                if "data" in output:
                    print("  <data>")
                    repr(output.data)
                    print("  </data>")
                if "metadata" in output and len(output.metadata)>0:
                    print("  <metadata>")
                    repr(output.metadata)
                    print("  </metadata>")
                print("</display>")
            elif output.output_type == "execute_result":
                print("<result>")
                if "data" in output:
                    print("  <data>")
                    print(output.data)
                    print("  </data>")
                if "metadata" in output and len(output.metadata)>0:
                    print("  <metadata>")
                    print(output.metadata)
                    print("  </metadata>")
                print("</result>")
            elif output.output_type == "error":
                print("<error>")
                print(output.ename)
                print(output.evalue)
                for frame in output.traceback:
                    print(frame)
                print("</error>")
        print("---------------------")

> python notebook
---------------------
cell 9d8a6aa0-8f58-4860-bcc1-2bfbdcb438b6 markdown
---------------------
# wordslab-notebooks-lib.jupyterlab

> Access wordslab-notebooks Jupyterlab extension version, curre
---------------------
cell 68f3493d-c252-4eb4-844b-abbd68ed3a70 markdown
---------------------
## Work together with AI in a Jupyterlab notebook - the Solveit method

A Jupyter notebook is a conv
---------------------
cell 0ff6fbdc-4a54-4e29-acbb-07529df8cfdd markdown
---------------------
## Install the Jupyterlab extension - wordslab-notebooks-lib

If you want to use "prompt" cells, you
---------------------
cell 65cd4cf8-d77b-4026-b428-bbd9550ea971 markdown
---------------------
## Communicate with the Jupyterlab extension
---------------------
cell ece4d545-8f78-4232-82fb-e837ea0185e4 code
---------------------
```python
#| export
import nbformat
```
---------------------
cell af2f3f45-f0da-4d37-9e39-b37d19ba5650 code
---------------------
```python
class JupyterlabNotebo